   * [Import Packages and Read credentials](#Import-Packages-and-Read-credentials)
   * [Read Before Proceeding](#Permission-to-access-information-from-these-networking-sites-via-automated-means-to-be-obtained-before-running-scraping-codes-included-in-this-section)
       * [Professional Networking site scraping example code](#Professional-Networking-Site)
           * [Inner Functions](#Professional-Networking-Site-Scraping-INNER-FUNCTIONS)
           * [Main Function (FUNCTION A)](#Professional-Networking-Site-Scraping-MAIN-FUNCTION)
       * [Company Review site scraping example code](#Company-Review-Site)
           * [Inner Functions](#Company-Review-Site-Scraping-INNER-FUNCTIONS)
           * [Main Function (FUNCTION B)](#Company-Review-Site-Scraping-MAIN-FUNCTION)
           
           
* [**START RUNNING HERE AFTER IMPORTING PACKAGES AND READING CREDENTIALS TO RUN CROSS REFERENCING CHECKS WITH SIMULATED DATA:** Example of Scraped data](#Example-of-scraped-data-to-input-as-data-for-testing-of-Company-background-online-presence-and-network-check)
* [Company background online presence and network check](#Company-background-online-presence-and-network-check)
    * [Inner Functions](#Company-background-online-presence-and-network-check-INNER-FUNCTIONS)
    * [Main Functions (FUNCTION C)](#Company-background-online-presence-and-network-check-MAIN-FUNCTION)

* [Scrape and Check Function](#Scrape-and-Check-Function)
    - *Scrape and Check Function* a function incorporating the functionality of reading credentials and using driver within same directory to scraping the relevant webpages and processing the checks with FUNCTIONS A, B and C 
    

This notebook demonstrates the possibility and logic to do a company background, online presence and network check in an effort to identify shell companies and prevent fraud transactions. Company information will be scraped from professional networking and company review sites and cross-referenced with google search results and some user-defined limit/minimal requirement (tweakable) to authenticate for entries to be genuine.

# Packages to be installed as instructed in README_CompanyCheck.txt and requirements_CompanyCheck.txt

In [ ]:
!pip install selenium
!pip install linkedin-scraper
!pip install pattern
!pip install bs4
!pip install itertools
!pip install textdistance
!pip install string
!pip install statistics
!pip install os
!pip install re
!pip install pandas
!pip install google-api-python-client
!pip install gensim

# Import Packages and Read credentials

In [1]:
import selenium
from linkedin_scraper import actions
from linkedin_scraper import Person
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import itertools
import textdistance
import string
from statistics import mean 
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import os
import re
from googleapiclient.discovery import build 
import pandas as pd
from gensim.utils import lemmatize
import en_core_web_sm
#!python -m spacy download en_core_web_sm
nlp = en_core_web_sm.load()

In [2]:
f = open("cred.txt", "r")
cred = (f.read())
email = cred.split("\n")[0]
password = cred.split("\n")[1]
ggl_api_key = cred.split("\n")[2]
ggl_cse_id = cred.split("\n")[3]

## Permission to access information from these networking sites via automated means to be obtained before running scraping codes included in this section

## Professional Networking Site

### Professional Networking Site Scraping INNER FUNCTIONS

In [ ]:
def makeUrl_LinkedIn_about(company_name):
   # company_name = 'Applied Materials'
    replaced_company_name = company_name.replace(' ', "-").lower()
    replaced_company_name

    return("https://www.linkedin.com/company/"+replaced_company_name+"/about/")

makeUrl_LinkedIn_about('Applied Materials')

In [ ]:
def makeUrl_LinkedIn_jobs(company_name):
   # company_name = 'Applied Materials'
    replaced_company_name = company_name.replace(' ', "-").lower()
    replaced_company_name

    return("https://www.linkedin.com/company/"+replaced_company_name+"/jobs/")

makeUrl_LinkedIn_jobs('Applied Materials')

In [ ]:
def extractDigit(string):
    return int(re.findall(r'\d+', string.replace(',', "").replace('', ""))[0])

### Professional Networking Site Scraping MAIN FUNCTION

In [ ]:
#Input: Linkedin email, linkedin password, company name
#Output: Dictionary of information

def linkedIn_name2dict(email,password,companyName):
    try:
        WINDOW_SIZE = "1920,1080"
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)

        driver = webdriver.Chrome(options=chrome_options)

        ### LOGIN ###
        actions.login(driver, email, password) #package function

        #INITIALISE STORAGE
        comp_linkedin_Info = {}


        ### ABOUT ###
        #GO TO COMPANY URL
        driver.get(makeUrl_LinkedIn_about(companyName))

        #Extract information

        #company linkedin employees
        num_EmployeesC = driver.find_element_by_css_selector('div.mt1')
        comp_linkedin_Info['Employees_on_Linkedin']  = int(re.findall(r'\d+', num_EmployeesC.text.replace(',', ""))[0])

        #the rest of details
        labels = driver.find_elements_by_tag_name("dt")
        labels.insert(3, 'Subsidiaries')
        values = driver.find_elements_by_tag_name("dd")

        numAttributes = min(len(labels), len(values))


        for i in range(numAttributes):
            try:
                value = values[i].text
            except:
                value = values[i]

            try:
                label = labels[i].text
            except:
                label = labels[i]

            #     print(label)
            #     print(value)
            #     print('')


            if label == 'Subsidiaries':
                comp_linkedin_Info[label] = value.split("subsidiaries: ")[1].split(" and 1 more.")[0].split(",")

            else:
                try:
                    comp_linkedin_Info[label] = extractDigit(value)
                    #100001 is max number 

                except:
                    comp_linkedin_Info[label] = value

        ### JOBS ###

        #GO TO COMPANY URL
        driver.get(makeUrl_LinkedIn_jobs('Applied Materials'))


        #Open Jobs
        num_jobs = driver.find_element_by_css_selector('h4.org-jobs-job-search-form-module__headline').text
        comp_linkedin_Info['Job Openings'] = extractDigit(num_jobs)
        comp_linkedin_Info['Name'] = companyName

        driver.quit()
        return comp_linkedin_Info
    
    except:
        try:
            driver.quit()
        except:
            pass
        print('Try using another set of credentials or variation of company name')

In [ ]:
#Test Function
comp_linkedin_Info = linkedIn_name2dict(email,password,"Applied Materials")
comp_linkedin_Info

## Company Review Site

### Company Review Site Scraping INNER FUNCTIONS

In [ ]:
def makeAchvDict(souped_splitted):

    year = 0
    achievement_dict = {}

    for x in souped_splitted:

        try :
            (int(x))
            new_year = x
            try:
                del achievement_dict[year][-1]
            except:
                pass
            year = new_year
            achievement_dict[year] = ['']
        except:
            #print('no')
            if x[0]!=",":
                achievement_dict[year][-1]+=(x.replace('\xa0', ""))
            else:
                achievement_dict[year].append('')
    del achievement_dict[year][-1]
    return achievement_dict

In [ ]:
def make_thousand(string):
    if string[-1]=='k':
        return int(float(string.replace('k', ""))*1000)
    else:
        return int(string)

In [ ]:
#Input: title and overview in webelement, text and cons in texts
#Output: List of lists. each inner list contains title, overview, pros and cons of 1 entry.

def reviews_compiler_page(titles,overview,pros_cons):
    company_reviews=[]
    pros_extr,cons_extr = pros_cons_extractor(pros_cons)
    for i in range(len(titles)):
        company_reviews.append([titles[i].text,
                               overview[i].text,
                               pros_extr[i], 
                               cons_extr[i]])
    return company_reviews


In [ ]:
#Input: bag of ambiguous elements that may contain pros/cons/replies from companies
#Output: return lists of pros and cons

def pros_cons_extractor(pros_cons):
    pros_list = []
    cons_list = []
    for i in range(len(pros_cons)):
        relevant_text = pros_cons[i]
        if "Pros\n" in relevant_text.text:
            pros_list.append(relevant_text.text.replace("Pros\n",""))
        elif "Cons\n" in relevant_text.text:
            cons_list.append(relevant_text.text.replace("Cons\n",""))
    return pros_list,cons_list
    

In [ ]:
##Input a list with strings with leading and trailing spaces
## Output: list with strings removed of leading and trailing spaces and lemmatized

def lemmatize_remove_space(alist):
    for i in range(len(alist)):
        alist[i] = ' '.join([wd.decode('utf-8').split('/')[0] for wd in lemmatize(alist[i])])
    return alist

lemmatize_remove_space([' Bats Are ! hanging by their feet', ' qwe ', 'zxc '])

In [ ]:
##Input a list with strings with captial letters and punctuations
## Output: join all strings into 1es

def proc_review(alist):
    return ' '.join(lemmatize_remove_space(alist))


proc_review(['bat be hang foot', 'qwe', 'zxc'])

In [ ]:
#Input: a list of fully preprocessed strings
#Output: mean score of similarity between each combination of reviews
def average_Sorensen_Dice_similarity(review_list_condensed):
    similarity_score = []
    for i in itertools.combinations(review_list_condensed, 2):
        #https://itnext.io/string-similarity-the-basic-know-your-algorithms-guide-3de3d7346227
        tokens_1 = i[0].split()
        tokens_2 = i[1].split()
        similarity_score.append(textdistance.sorensen(tokens_1 , tokens_2))
    return mean(similarity_score)

### Company Review Site Scraping MAIN FUNCTION

In [ ]:
#Input: company name
#Output: Dictionary of information

def glassdoor_name2dict(email,password,company_name_search_input):
    try:

        #driver = webdriver.Chrome()
        #driver = webdriver.Chrome(options=chrome_options)

        WINDOW_SIZE = "1920,1080"
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)

        driver = webdriver.Chrome(options=chrome_options)


        driver.get('https://www.glassdoor.sg/member/home/companies.htm')

        #Initialise output
        comp_glassdoor_Info={}


        ### Log In ###
        #Input cred
        email_box = driver.find_element_by_id('userEmail')
        pw_box = driver.find_element_by_id('userPassword')
        email_box.send_keys(email)
        pw_box.send_keys(password)



        #click log in button
        login_button = driver.find_element_by_css_selector('button.gd-ui-button.minWidthBtn.css-8i7bc2')
        login_button.click()

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'sc.keyword')))

        #Access SearchBox
        s_box = driver.find_element_by_id('sc.keyword')
        s_box.send_keys(company_name_search_input)

        #Click on search
        search_button = driver.find_element_by_css_selector('button.gd-ui-button.ml-std.col-auto.search__SearchStyles__newSearchButton.css-iixdfr')
        search_button.click()
        
        #Get company website
        comp_ws = driver.find_element_by_css_selector("p.webInfo.mb-0.mt-xxsm").text
        comp_glassdoor_Info['Website'] = comp_ws

        #Access 1st result and create URL for company
        boxed = driver.find_element_by_css_selector('div.col-3.logo-and-ratings-wrap')
        messy = boxed.get_attribute('innerHTML')
        partURL = messy.split('="')[1].split('" ')[0]
        fullURL = 'https://www.glassdoor.sg'+partURL

        #To company URL
        driver.get(fullURL)


        ### Reached Company page ###
        try:
            #See more achievements
            seeMoreAchv = driver.find_element_by_css_selector('button.d-flex.css-9k4uin.e18lin5w0')
            seeMoreAchv.click()


            #get all achievements
            allAchvs = driver.find_element_by_class_name('css-1igu9t8.e19w1ug90')
            achv_html = allAchvs.get_attribute('innerHTML')
            achv_html_soup = BeautifulSoup(achv_html)
            achv_souped_splitted = (achv_html_soup.get_text('\n')).splitlines()
            achvDict = makeAchvDict(achv_souped_splitted)

            #Add achvdict to output dict
            comp_glassdoor_Info['Awards & Accolades'] = achvDict
        except:
            comp_glassdoor_Info['Awards & Accolades'] = {}


        #CEO info
        ceo_box = driver.find_element_by_css_selector('div.d-lg-table-cell.ceoName.pt-sm.pt-lg-0.px-lg-sm')
        comp_glassdoor_Info['CEO name']  = ceo_box.text.split('\n')[0]
        comp_glassdoor_Info['CEO ratings'] = ceo_box.text.split('\n')[1]

        #get number of reviews
        reviews_box = driver.find_element_by_css_selector('a.eiCell.cell.reviews')
        comp_glassdoor_Info['Company Reviews'] = int(make_thousand(reviews_box.text.split('\n')[0]))

        #get number of jobs
        jobs_box = driver.find_element_by_css_selector('a.eiCell.cell.jobs')
        comp_glassdoor_Info['Jobs'] = int(make_thousand(jobs_box.text.split('\n')[0]))

        #get number of salaries
        sal_box = driver.find_element_by_css_selector('a.eiCell.cell.salaries')
        comp_glassdoor_Info['Salaries'] = int(make_thousand(sal_box.text.split('\n')[0]))

        #get number of interview
        interview_box = driver.find_element_by_css_selector('a.eiCell.cell.interviews')
        comp_glassdoor_Info['Interview'] = int(make_thousand(interview_box.text.split('\n')[0]))

        #get number of benefit
        benefit_box = driver.find_element_by_css_selector('a.eiCell.cell.benefits')
        comp_glassdoor_Info['Benefit'] = int(make_thousand(benefit_box.text.split('\n')[0]))

        ### Go to reviews
        #Access 1st result and create URL for review
        review_box = driver.find_element_by_css_selector('a.eiCell.cell.reviews ')
        review_url = review_box.get_attribute('href')

        #To company URL
        driver.get(review_url)

        ############################Get reviews within page boundary of 5 (50 reviews)
        user_page_limit=5
        review_pages = []

        next_page = True
        while next_page:
            titles =driver.find_elements_by_class_name('reviewLink')
            overview = driver.find_elements_by_css_selector("p.mainText.mb-0")
            pros_cons = driver.find_elements_by_css_selector("div.v2__EIReviewDetailsV2__fullWidth ")

            review_pages += reviews_compiler_page(titles,overview,pros_cons)

            #Read current page and decide if go next page
            reviewTotalInfo = driver.find_element_by_css_selector('div.paginationFooter').text.split(" of ")
            now_page = int(reviewTotalInfo[0].split(" ")[-1])
            limit_page = int(reviewTotalInfo[1].split(" ")[0].replace(",",""))
            if (now_page < limit_page) and (now_page/10 <user_page_limit) :
                #find next button and click
                next_button = driver.find_element_by_class_name('nextButton.css-sed91k')
                next_button.click()
                try:
                    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'a.reviewLink')))
                except:
                    driver.get(driver.current_url)
            else:
                next_page = False
        ############################

        #create a list where an element is 1 review:
        review_list_condensed = [proc_review(alist) for alist in review_pages]

        #Add review similarity score to  output dict
        comp_glassdoor_Info['Review similarity score'] = average_Sorensen_Dice_similarity(review_list_condensed)
        comp_glassdoor_Info['Name'] = company_name_search_input
        driver.quit()
        return comp_glassdoor_Info

    except:
        try:
            driver.quit()
        except:
            pass
        print('Try using another set of credentials or variation of company name')

In [ ]:
#Test Function
comp_glassdoor_Info = glassdoor_name2dict(email,password,"Applied Materials")
comp_glassdoor_Info

# Example of scraped data to input as data for testing of Company background online presence and network check

In [3]:
comp_linkedin_Info = {'Employees_on_Linkedin': 21714,
 'Website': 'http://www.appliedmaterials.com',
 'Industry': 'Semiconductors',
 'Company size': 10001,
 'Subsidiaries': ['Applied Materials - Israel',
  'Applied Materials India',
  'Applied Materials South East Asia'],
 'Headquarters': 'Santa Clara, CA',
 'Type': 'Public Company',
 'Specialties': 'Semiconductor Manufacturing Equipment, Materials Engineering, Flat Panel Display Manufacturing Equipment, and Global Services for the industries served',
 'Job Openings': 1028,
  'Name': 'Applied Materials'}

In [4]:
comp_glassdoor_Info = {'Awards & Accolades': {'2015': ['Fortune 500,Fortune'],
  '2014': ["World's Most Admired Companies,Fortune Magazine"],
  '2013': ["2013 World's Most Ethical Companies,Ethisphere Institute",
   'Preferred Quality Supplier (PQS) Award,Intel Corporation',
   'Energy Innovator Award,Silicon Valley Power',
   'Best Companies to Work For,Silicon India',
   'Corporate Innovation Recognition Award,IEEE'],
  '2012': ['Top Greenest Companies,NEWSWEEK',
   'Top Military Friendly Employer,G.I. Jobs'],
  '2011': ['Service Leadership Award,City Year San Jose/Silicon Valley',
   'Technovation Semiconductor Ecosystem Award,India Semiconductor Association (ISA)',
   '100 Best Places to Work in Information Technology,ComputerWorld',
   'Top Corporate Philanthropist,Silicon Valley/San Jose Business Journal',
   '100 Best Corporate Citizens,Corporate Responsibility Magazine'],
  '2010': ['Supplier Excellence Award,Texas Instruments',
   'EuroAsia IC Industry Subsystems & Components Award,EuroAsia Semiconductor Magazine',
   'Best Places to Work for LGBT Equality,Human Rights Campaign'],
  '2009': ['Top 100 Best Places to Work in IT,ComputerWorld',
   "Barron's 500,Barron's",
   'Top 500 Green Companies,Newsweek'],
  '2008': ['World’s Most Admired Companies,Fortune',
   'Training Top 125,Training']},
 'CEO name': 'Gary Dickerson',
 'CEO ratings': '573 Ratings',
 'Company Reviews': 1900,
 'Jobs': 803,
 'Salaries': 7500,
 'Interview': 473,
 'Benefit': 531,
 'Review similarity score': 0.31225144345500194,
 'Name': 'Applied Materials',
  'Website': 'http://www.appliedmaterials.com'}

# Company background online presence and network check

### Company background, online presence and network check INNER FUNCTIONS

In [5]:
##Input: glassdoor dictionary
##Output: number of awards

def num_awards(comp_glassdoor_Info):
    award_dict = comp_glassdoor_Info['Awards & Accolades']
    total_awards = 0
    for i, v in award_dict.items():
        total_awards+=len(v)
    return total_awards

In [6]:
##Input: glassdoor dictionary
##Output: list of search items for awards

def award_check_intermediate(comp_glassdoor_Info):
    result = []
    award_dict = comp_glassdoor_Info['Awards & Accolades']
    total_awards = 0
    for i, v in award_dict.items():
        if len(v)>1:
            for vv in v:
                result.append((i + ' ' + vv).replace(","," "))
        else:
            result.append((i + ' ' + v[0]).replace(","," "))
    return [result, comp_glassdoor_Info['Name']]


#award_check_intermediate({'Awards & Accolades':{},'Name':'asd'})
award_check_intermediate(comp_glassdoor_Info)

[['2015 Fortune 500 Fortune',
  "2014 World's Most Admired Companies Fortune Magazine",
  "2013 2013 World's Most Ethical Companies Ethisphere Institute",
  '2013 Preferred Quality Supplier (PQS) Award Intel Corporation',
  '2013 Energy Innovator Award Silicon Valley Power',
  '2013 Best Companies to Work For Silicon India',
  '2013 Corporate Innovation Recognition Award IEEE',
  '2012 Top Greenest Companies NEWSWEEK',
  '2012 Top Military Friendly Employer G.I. Jobs',
  '2011 Service Leadership Award City Year San Jose/Silicon Valley',
  '2011 Technovation Semiconductor Ecosystem Award India Semiconductor Association (ISA)',
  '2011 100 Best Places to Work in Information Technology ComputerWorld',
  '2011 Top Corporate Philanthropist Silicon Valley/San Jose Business Journal',
  '2011 100 Best Corporate Citizens Corporate Responsibility Magazine',
  '2010 Supplier Excellence Award Texas Instruments',
  '2010 EuroAsia IC Industry Subsystems & Components Award EuroAsia Semiconductor Maga

In [7]:
##Input: query, api key, cse key
##Output: List of dictionaries, each dict a result 

def google_query(query, ggl_api_key, ggl_cse_id, **kwargs):
    query_service = build("customsearch", 
                          "v1", 
                          developerKey=ggl_api_key
                          )  
    query_results = query_service.cse().list(q=query,    # Query
                                             cx=ggl_cse_id,  # CSE ID
                                             **kwargs    
                                             ).execute()
    return query_results



In [8]:
##Input: company info in dictionary from scraping
##Output: authentication from google searches

def check_top10_google_results_allAwards(comp_glassdoor_Info):
    awards, comp_name = award_check_intermediate(comp_glassdoor_Info)
    
    authenticity = False

    for award in awards:
        if not authenticity: #if non of the award has yet to be authenticated
            this_Award_t10result = google_query(award+' '+comp_name,
                          ggl_api_key, 
                          ggl_cse_id, 
                          num = 10
                          ) #get top 10 result of this award

            if int(this_Award_t10result['queries']['request'][0]['totalResults'])>0:
                listo_results = this_Award_t10result['items']

                for result in listo_results:
                #a single search result on i-th award
                    #check source
                    source_link = result['pagemap']['metatags'][0]['og:url']
                    source_siteCheck1 = 'glassdoor' not in  source_link# T/F returns if from glassdoor (the source we are trying to verify)
                    source_siteCheck2 = 'linkedin' not in  source_link
                    source_siteCheck = source_siteCheck1 & source_siteCheck2

                    #check contents
                    result_summary = (result['title'] + ' ' + result['snippet']).replace("\n", " ") #header and contents
                    #
                    ##T/F if company is mentioned
                    company_name_relevance = comp_name.lower() in result_summary.lower() 
                    #
                    ##T/F if award is mentioned
                    awardInfo = set(award.translate(str.maketrans('', '', string.punctuation)).lower().split(" "))
                    searchInfo = set(result_summary.translate(str.maketrans('', '', string.punctuation)).lower().split(" "))
                    #
                    search_contentCheck = (awardInfo<=searchInfo)

                    if search_contentCheck & source_siteCheck:
                        authenticity = True
                        break

    return  authenticity    
                
#check_top10_google_results_allAwards({'Awards & Accolades':{},'Name':'asd'})                
check_top10_google_results_allAwards(comp_glassdoor_Info)                 

True

In [9]:
##Input: company info in dictionary from scraping and cut-off for minimal employees nodes on linkedin default at 100
##Output: authentication from google searches in boolean

def check_networkEmployees(comp_linkedin_Info, n1):
    return (comp_linkedin_Info['Employees_on_Linkedin'] >=n1)

    
check_networkEmployees(comp_linkedin_Info, 100)    

True

In [10]:
##Input: company info in dictionary from scraping and cut-off for minimal job openings on linkedin and glassdoor default at 20
##Output: if job openings above cut-off in boolean

def check_jobs(comp_linkedin_Info, comp_glassdoor_Info, n3):
    TF = (comp_linkedin_Info['Job Openings'] >=n3) & (comp_glassdoor_Info['Jobs'] >=n3) 
    return TF

check_jobs(comp_linkedin_Info,comp_glassdoor_Info, 20)    

True

In [11]:
##Input: company info in dictionary from scraping and limit for review similarity at default value of 50%
##Output: company reviews are below limit in boolean

def check_reviews(comp_glassdoor_Info, n2):
    TF = (comp_glassdoor_Info['Review similarity score']<=n2)
    return TF

check_reviews(comp_glassdoor_Info, 0.5)    

True

In [12]:
##Input: company info in dictionaries from scraping of diff sites
##Output: if company official sites are the same in boolean

def check_website(comp_glassdoor_Info, comp_linkedin_Info):
    TF = (comp_glassdoor_Info['Website']==comp_linkedin_Info['Website'])
    return TF

check_website(comp_glassdoor_Info, comp_linkedin_Info)    

True

In [13]:
##Input: dictionary from scraping
##Output: authentication CEO of company from google searches

def check_CEO(comp_glassdoor_Info):
    comp_name = award_check_intermediate(comp_glassdoor_Info)[1]
    CEO_name = comp_glassdoor_Info['CEO name']
    
    authenticity = False


    CEO_t10result = google_query('Who is the CEO of '+comp_name,
                  ggl_api_key, 
                  ggl_cse_id, 
                  num = 10
                  ) #get top 10 result of this award

    if int(CEO_t10result['queries']['request'][0]['totalResults'])>0:
        listo_results = CEO_t10result['items']

        for result in listo_results:
        #a single search result on i-th award
            #check source
            try:
                source_link = result['pagemap']['metatags'][0]['og:url']
                source_siteCheck1 = 'glassdoor' not in  source_link# T/F returns if from glassdoor (the source we are trying to verify)
                source_siteCheck2 = 'linkedin' not in  source_link
                source_siteCheck = source_siteCheck1 & source_siteCheck2
                
                #check contents
                result_summary = (result['title'] + ' ' + result['snippet']).replace("\n", " ") #header and contents

                ##T/F if company is mentioned
                company_name_relevance = comp_name.lower() in result_summary.lower() 
                #
                ##T/F if CEO name is mentioned
                CEO_Info = set(CEO_name.lower().split(" "))
                searchInfo = set(result_summary.translate(str.maketrans('', '', string.punctuation)).lower().split(" "))
                #
                search_contentCheck = (CEO_Info<=searchInfo)


                if search_contentCheck & source_siteCheck:
                    authenticity = True
                    break
            except:
                pass

    return  authenticity    

check_CEO(comp_glassdoor_Info)                 

True

### Company background online presence and network check MAIN FUNCTION

#### Function to give level of credibility according to background check

Possible output:
- 'Fail to Authenticate'
- 'Low'
- 'Mid'
- 'High'

In [14]:
##Input: dictionaries from scraping
##Output: Credibility classification

def companyBackground_onlinePresenceAndNetwork_check(comp_linkedin_Info,comp_glassdoor_Info, n1,n2,n3):
    #check if official company website is the same 
        ###check_website(comp_glassdoor_Info, comp_linkedin_Info)###
        
    #check review similiarity 
        ###check_reviews(comp_glassdoor_Info, cut_off =0.5)###
    
    #check job openings
        ###check_jobs(comp_linkedin_Info,comp_glassdoor_Info, cut_off =20)###    
    
    #check linkedin employee base size
        ###check_networkEmployees(comp_linkedin_Info, cut_off =100)###
    
    #check google search result for award recipient authentication
        ###check_top10_google_results_allAwards(comp_glassdoor_Info)###
        
        
    #check CEO from alternate source using google search
        ###check_CEO(comp_glassdoor_Info)###
            
    #####################
    
    site_Check = check_website(comp_glassdoor_Info, comp_linkedin_Info)   
    ceo_Check = check_CEO(comp_glassdoor_Info)  
    
    employeeNetwork_Check = check_networkEmployees(comp_linkedin_Info, n1)
    awards_Check = check_top10_google_results_allAwards(comp_glassdoor_Info)
    
    reviews_Check = check_reviews(comp_glassdoor_Info, n2)
    jobOpenings_Check = check_jobs(comp_linkedin_Info,comp_glassdoor_Info,n3)
    
    #####################
    cred_failTOlow = site_Check & ceo_Check
    
    cred_lowTOmid = cred_failTOlow & employeeNetwork_Check & awards_Check
    
    cred_midTOhigh = cred_lowTOmid & reviews_Check & jobOpenings_Check
    
    credibility = 'Fail to Authenticate'
    
    if cred_midTOhigh:
        credibility = 'High'
    elif cred_lowTOmid:
        credibility = 'Mid'
    elif cred_failTOlow:
        credibility = 'Low'
    else:
        credibility = 'Fail to Authenticate'
    
    return credibility

companyBackground_onlinePresenceAndNetwork_check(comp_linkedin_Info,comp_glassdoor_Info, n1 = 100, n2 = 0.5, n3=20)

'High'

# Scrape and Check Function

This function runs all the functions aforementioned with 
- Input: Company name
- Output: Credibility classification

In [15]:
def cred_check(company_name, n1 = 100, n2 = 0.5, n3=20):
    
    #credentials
    f = open("cred.txt", "r")
    cred = (f.read())
    email = cred.split("\n")[0]
    password = cred.split("\n")[1]
    ggl_api_key = cred.split("\n")[2]
    ggl_cse_id = cred.split("\n")[3]
    
    #company info
    comp_linkedin_Info = linkedIn_name2dict(email,password,company_name)
    comp_glassdoor_Info = glassdoor_name2dict(email,password,company_name)
    
    #check
    return companyBackground_onlinePresenceAndNetwork_check(comp_linkedin_Info,comp_glassdoor_Info,n1,n2,n3)

In [ ]:
cred_check('Applied Materials')